In [23]:
from __future__ import print_function
import os.path
import httplib2
#import apiclient.discovery
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from datetime import datetime
import pandas as pd


# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ'

def get_df(data):
    data_dict = {}
    cols = data[0]
    for col in cols:
        data_dict[col] = []
    
    for i in range(len(cols)):
        for k in range(1, len(data)):
            data_dict[cols[i]].append(data[k][i])
    
        
    data_df = pd.DataFrame.from_dict(data_dict)
    return data_df


"""Shows basic usage of the Sheets API.
Prints values from a sample spreadsheet.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secret_852918827851-jphlh48nvfk0rairmigc9vm4kbaaeeeb.apps.googleusercontent.com.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range='managers').execute()
managers = result.get('values', [])

result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range='leads').execute()
leads = result.get('values', [])


managers_df = get_df(managers)
leads_df = get_df(leads)



In [25]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', 5)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199
leads_df


,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1
...,...,...,...,...,...,...
3335,ffc0b363-2ae9-11eb-ac1f-c412f533dba1,2020-11-20 08:36:48,cpc,instagram,ad52c7a8-a752-11e7-8115-c412f533dba1,00000000-0000-0000-0000-000000000000
3336,ffcdebc8-15c2-11eb-ac1d-c412f533dba1,2020-10-24 10:34:43,utm_medium,utm_source,ad52c7a8-a752-11e7-8115-c412f533dba1,f3357d13-15c7-11eb-ac1d-c412f533dba1


In [35]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', 50)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199
leads_df.groupby('l_manager_id').count()


,lead_id,created_at,d_utm_medium,d_utm_source,l_client_id
l_manager_id,,,,,
00000000-0000-0000-0000-000000000000,491,491,491,491,491
0c5d545d-06ff-11eb-ac18-c412f533dba1,1,1,1,1,1
0d789282-f8ae-11ea-ac18-c412f533dba1,2,2,2,2,2
1bb0e7ca-5892-11ea-abeb-c412f533dba1,1,1,1,1,1
1e9c5562-0cfc-11ea-abe1-c412f533dba1,280,280,280,280,280
2346423e-adcf-11e2-921f-002522b9dae9,1,1,1,1,1
31298620-0355-11eb-ac18-c412f533dba1,1,1,1,1,1
31d3f905-0c7c-11eb-ac1a-c412f533dba1,1,1,1,1,1
43756fa4-57a0-11e9-ab9a-c412f533dba1,341,341,341,341,341


In [27]:
channels = leads_df['d_utm_source'].unique()
print(channels)

['viber' 'instagram' '' 'vk' 'sms' 'google' 'utm_source' 'insta'
 'vkontakte' 'yandex' 'ycard' 'ycard#!/tproduct/225696739-1498486363994'
 'ig']


In [29]:
managers_df = managers_df.rename(columns={"manager_id": 'l_manager_id'})
managers_df

,l_manager_id,d_manager,d_club
0,1e9c5562-0cfc-11ea-abe1-c412f533dba1,manager #8,club #2
1,43756fa4-57a0-11e9-ab9a-c412f533dba1,manager #11,club #4
...,...,...,...
11,e463b03f-2d74-11eb-ac1f-c412f533dba1,manager #3,club #1
12,f44596c6-12cd-11eb-ac1c-c412f533dba1,manager #6,club #2


In [30]:
leads_with_clubs = pd.merge(leads_df, managers_df, how='inner', on=['l_manager_id'])

In [31]:
leads_with_clubs

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id,d_manager,d_club
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2
1,0127f039-0541-11eb-ac18-c412f533dba1,2020-10-03 10:23:52,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,b7b746d1-1adf-11e8-812f-c412f533dba1,manager #8,club #2
...,...,...,...,...,...,...,...,...
2769,fe8be0d9-2690-11eb-ac1d-c412f533dba1,2020-11-14 19:49:36,cpc,yandex,abed5496-ca88-11e9-abb8-c412f533dba1,15fdbdc4-2713-11eb-ac1d-c412f533dba1,manager #5,club #4
2770,07d2def4-2eb3-11eb-ac1f-c412f533dba1,2020-11-25 04:13:23,social,instagram,e463b03f-2d74-11eb-ac1f-c412f533dba1,8bcbbe39-2eee-11eb-ac1f-c412f533dba1,manager #3,club #1
